###### Description: In this notebook we read landlord state rows from incoming csv files into a streamig dataframe, transform (clean, cast, rename) the data, add/update the latest state to a static hive tabe
###### Objective: (incoming csv files) --> "landlord_streamingDF" --> "landlord_df" --> "landlord_data"

In [2]:
import requests
import json
import optimus as op
import phonenumbers 
import re
import datetime

from pyspark.sql.types import StringType, IntegerType, TimestampType, DateType, DoubleType, StructType, StructField
from pyspark.sql.functions import udf
from pyspark.sql import SparkSession
from pyspark.sql import HiveContext
from pyspark.sql import SQLContext, Row
from pyspark.sql.functions import unix_timestamp, from_unixtime
from pyspark.sql import functions as F
from pyspark.sql.window import Window as W
from functools import reduce  # For Python 3.x
from pyspark.sql import DataFrame
from pyspark.sql.functions import lit
from pyspark.sql.functions import rank, col
import time

In [3]:
# Schema for Landlord JSON
landlord_schema = StructType([
            StructField("Landlord_id", IntegerType(), False),
            StructField("Password", StringType(), True),
            StructField("Landlord_name", StringType(), False),
            StructField("Address_line_1", StringType(), False),
            StructField("City", StringType(), False),
            StructField("Post_code", StringType(), True),
            StructField("Region", StringType(), True),
            StructField("event_time", TimestampType(), True),
            StructField("fetch_time", StringType(), True)])

# Schema for building JSON
building_schema = StructType([
            StructField("Building_name", StringType(), True),
            StructField("Landlord_id", IntegerType(), False),
            StructField("Address_line_1", StringType(), False),
            StructField("City", StringType(), False),
            StructField("Post_code", StringType(), True),
            StructField("Region", StringType(), True)])

# Schema for Apartment JSON
apartment_schema = StructType([
            StructField("Apartment_number", IntegerType(), True),
            StructField("Type", StringType(), True),
            StructField("Rent_fee", StringType(), True),
            StructField("Building_name", StringType(), True),
            StructField("Appt_details", StringType(), True)])

# Schema for Contractor
contractor_schema = StructType([
            StructField("Contract_id", IntegerType(), False),
            StructField("Name", StringType(), True),
            StructField("Address_line_1", StringType(), False),
            StructField("City", StringType(), False),
            StructField("Post_code", StringType(), True),
            StructField("Region", StringType(), True)])

# Schema for Tenant
tenant_schema = StructType([
            StructField("Tenant_id", IntegerType(), False),
            StructField("First_name", StringType(), True),
            StructField("Last_name", StringType(), False),
            StructField("Ssn", StringType(), True),
            StructField("Phone", StringType(), True),
            StructField("Email", StringType(), True), 
            StructField("Mobile", StringType(), True)])

# Schema for Lease 
lease_schema = StructType([
            StructField("Lease_id", IntegerType(), False),
            StructField("Start", StringType(), True),
            StructField("End", StringType(), False),
            StructField("Deposit", StringType(), True),
            StructField("Tenant_id", IntegerType(), True),
            StructField("Apartment_id", IntegerType(), True)])

# Schema  for Rent
rent_schema = StructType([
            StructField("Rent_id", IntegerType(), False),
            StructField("Rent_fee", StringType(), True),
            StructField("Late_fee", StringType(), False),
            StructField("Due_date", TimestampType(), True),
            StructField("Lease_id", IntegerType(), True),
            StructField("Pay_id", IntegerType(), True)])

# Schema for Payment
payment_schema = StructType([
            StructField("Payment_id", IntegerType(), False),
            StructField("Pay_date", TimestampType(), True),
            StructField("Pay_amount", StringType(), False),
            StructField("Method", StringType(), True),
            StructField("Rent_id", IntegerType(), True)])

# Schema for Apartment Maintenance
apt_maintenance_schema = StructType([
            StructField("Maintenance_id", IntegerType(), False),
            StructField("Apartment_number", IntegerType(), True),
            StructField("Mdate", StringType(), False),
            StructField("Issue_reported", StringType(), True),
            StructField("Contractor_id", IntegerType(), True), 
            StructField("Resolution", StringType(), True), 
            StructField("Status", StringType(), True),
            StructField("Charges_incurred", StringType(), True)])

# Schema for Building Maintenance
building_maintenance_schema = StructType([
            StructField("Maintenance_id", IntegerType(), False),
            StructField("Building_name", StringType(), True),
            StructField("Ndate", StringType(), False),
            StructField("Issue_reported", StringType(), True),
            StructField("Contractor_id", IntegerType(), True), 
            StructField("Resolution", StringType(), True), 
            StructField("Status", StringType(), True)])


In [4]:
# Function to get SparkDataFrame after reading JSON data from API
def getSparkDataFrame(url, schema):
  appdf = requests.get(url)
  objJSON = appdf.json()
  a=[json.dumps(objJSON)]
  jsonRDD = sc.parallelize(a)
  df = spark.read.schema(schema).json(jsonRDD)
  return df

# convert string value to Float value
def string_to_float(x):
  return float(x[1:])

# Get DataFrame without new Line characters
# Especially for Apartment, ApartmentMaintenance

def getSparkDataFrameWithoutLFChar(url, schema):
  appdf = requests.get(url)
  str=''
  for line in appdf.iter_lines():
    str = line.decode(encoding='utf-8', errors='strict')
    # escaping \n works for python3, if it's python 2 no need to escape
    str = str.replace('\\n', '')
  json_str=json.loads(str)
  df = spark.createDataFrame(json_str, schema)
  return df


In [5]:
def unionAll(*dfs):
  return reduce(DataFrame.unionAll, dfs)

In [6]:
def fixTenantRow(c):
    # get the Mobile field
    number = c.Mobile

    # initialize variables 
    is_valid_number = "N"
    clean_number = None
    number_type = None
    valid_mail = None

    p = None

    if number is not None:
        # Clean the Mobile Number first
        try:
            p = phonenumbers.parse(number, None)

            if phonenumbers.is_valid_number(p):
                is_valid_number = "Y"
            elif phonenumbers.truncate_too_long_number(p):
                is_valid_number = "Y"
            else:
                is_valid_number = "N"

            clean_number = "%s%s" % (p.country_code, p.national_number)
            
        except:
            p = None

    # clean up PhoneNumber
    phone_no = c.Phone
    if phone_no is not None:
      phone_no = phone_no.replace(' ', '')
      if (len(phone_no) != 10):
        phone_no = None
    
    # validate Email 
    if re.match(r"^[A-Za-z0-9\.\+_-]+@[A-Za-z0-9\._-]+\.[a-zA-Z]*$", c.Email):
      valid_mail = c.Email
    
    return Row( 
		Tenant_id = c.Tenant_id,
		First_name = c.First_name,
		Last_name = c.Last_name,
		Ssn = c.Ssn,
		Phone = phone_no,
		Email = valid_mail,
        Mobile=clean_number 
    )

In [7]:
# function validating Post Code  
def validatePostCode(postCode):
  if (re.match(r"^[0-9]{5}(-[0-9]{4})?$", postCode)):
    return postCode
  else:
    return None

In [8]:
# Create function to check each row if it exists in db
def checkIsRowExists(c, table_df):
  
  existing_row = table_df.filter(table_df.Landlord_id.isin(c.Landlord_id)) 
  if (existing_row.count() > 0):
    return Row(
        landlord_seq = existing_row.landlord_seq,
        Landlord_id = c.Landlord_id,
        Password = c.Password,
        Landlord_name = c.Landlord_name,
        Address_line_1 = c.Landlord_name,
        City = c.City,
        Post_code = c.Post_code,
        Region = c.Region
    )
  else:
    return Row(
      landlord_seq = null,
      Landlord_id = null,
      Password = c.Password,
      Landlord_name = c.Landlord_name,
      Address_line_1 = c.Landlord_name,
      City = c.City,
      Post_code = c.Post_code,
      Region = c.Region
    )

In [9]:
# Functions to udf
udfstring_to_float = udf(string_to_float, StringType())
# UDF for validatePostCode function  
udfValidatePostCode = udf(validatePostCode, StringType())

###### Description: Get landlord csv files as a streaming "landlord_streamingDF" and process it on the fly and get transformed stream "landlord_df"
###### Objective: (incoming csv files) --> "landlord_streamingDF" --> "landlord_df"

In [11]:
# Get Landlord Steaming DataFrame from csv files

# streaming starts here by reading the input files 
landlord_Path = "/FileStore/apartment/landlord/inprogress/"
landlord_streamingDF = (
  spark
    .readStream
    .schema(landlord_schema)
    .option("maxFilesPerTrigger", "1")
    .option("header", "true")
    .csv(landlord_Path)
)

landlord_df = landlord_streamingDF.withColumn("Post_code", udfValidatePostCode("Post_code") )
# landlord_df = landlord_df.select(landlord_df.Landlord_id, landlord_df.Password, landlord_df.Landlord_name, landlord_df.Address_line_1,  landlord_df.City, landlord_df.PostCode, landlord_df.Region, landlord_df.Seen)
# Instantiation of DataTransformer class:
transformer = op.DataFrameTransformer(landlord_df)
# Replace NA with 0's
transformer.replace_na(0.0, columns="*")
# Clear accents: clear_accents only from name column and not everywhere 
transformer.clear_accents(columns='*')
# Remove special characters:  From all Columns 
transformer.remove_special_chars(columns=['Landlord_name', 'Address_line_1', 'City', 'Region'])


- ###### Now "landlord_df" contains pre-processed landlord state rows
- ###### After this point we need comparison
- ###### Stream-Stream subtraction is not supported
- ###### So we dump the incoming data to a query result "landlord_datalake" which will give updated resulsts upon request
- ###### "landlord_datalake" is not streaming but it will give updated results upon request
- ###### From "landlord_datalake" we filter out the unseen rows to "unseen_landlord_df"

In [13]:
landlord_datalake_query = landlord_df.writeStream.format("memory").queryName("landlord_datalake").start()

## Enter batch mode

###### Take a snapshot of the landlord_df where Seen = false
###### Add the fetch time columns to hive table landlord_seq_tracker

In [16]:
# OLD Code
# def getDelta_df(entity):
  
#   #   Save snapshot of data into hive table to work with
#   if (len(spark.sql("SHOW TABLES LIKE '" + entity + "_temp'").collect()) == 1):
#       spark.sql("drop table " + entity + "_temp")
#       spark.sql("select * from " + entity + "_datalake").write.saveAsTable(entity + "_temp")
#   else:
#       spark.sql("select * from " + entity + "_datalake").write.saveAsTable(entity + "_temp")
      
#   #   Take snapshot
#   datalake_snapshot = spark.sql("select * from " + entity + "_temp")
    
#   if (len(spark.sql("SHOW TABLES LIKE '" + entity + "_tracker'").collect()) == 1):
#     seq_tracker = spark.sql("select * from " + entity + "_tracker")
#     datalake_eq = (( datalake_snapshot
#                     .join(seq_tracker, seq_tracker.sequence == datalake_snapshot.fetch_time))
#                    .drop("sequence").write.saveAsTable("temp_data"))

#     spark.sql("refresh table temp_data")

#     delta_df = datalake_snapshot.subtract(spark.sql("select * from temp_data"))

#     if (len(spark.sql("SHOW TABLES LIKE '" + entity + "_delta" + "'").collect()) == 1):
#       spark.sql("drop table " + entity + "_delta")
#       delta_df.write.saveAsTable(entity + "_delta")
#     else:
#       delta_df.write.saveAsTable(entity + "_delta")
      
#     delta_df.select(col("fetch_time").alias("sequence")).distinct().write.insertInto(entity + "_tracker")

#     spark.sql("drop table temp_data")
#   else:
#     datalake_snapshot.write.saveAsTable(entity + "_delta")
#     datalake_snapshot.select(col("fetch_time").alias("sequence")).distinct().write.saveAsTable(entity + "_tracker")    
  
#   return spark.sql("select * from " + entity + "_delta")

In [17]:
def getDelta_df(entity):
  
  #   Save snapshot of data into hive table to work with
  spark.sql("select * from " + entity + "_datalake").write.mode("overwrite").saveAsTable(entity + "_temp")
  #   Take snapshot
  datalake_snapshot = spark.sql("select * from " + entity + "_temp")
    
  if (len(spark.sql("show tables like '" + entity + "_tracker'").collect()) == 1):
    seq_tracker = spark.sql("select * from " + entity + "_tracker")
    datalake_eq = (( datalake_snapshot
                    .join(seq_tracker, seq_tracker.sequence == datalake_snapshot.fetch_time))
                   .drop("sequence").write.saveAsTable("temp_data"))

    spark.sql("refresh table temp_data")

    delta_df = datalake_snapshot.subtract(spark.sql("select * from temp_data"))
    
    delta_df.write.mode("overwrite").saveAsTable(entity + "_delta")
      
    delta_df.select(col("fetch_time").alias("sequence")).distinct().write.insertInto(entity + "_tracker")

    spark.sql("drop table temp_data")
  else:
    datalake_snapshot.write.saveAsTable(entity + "_delta")
    datalake_snapshot.select(col("fetch_time").alias("sequence")).distinct().write.saveAsTable(entity + "_tracker")    
  
  return spark.sql("select * from " + entity + "_delta")

In [18]:
def resetTrackingData(entity):
  if (len(spark.sql("show tables like '" + entity + "_delta'").collect()) == 1):
      spark.sql("drop table " + entity + "_delta")
      
  if (len(spark.sql("show tables like '" + entity + "_temp'").collect()) == 1):
      spark.sql("drop table " + entity + "_temp")
      
  if (len(spark.sql("show tables like '" + entity + "_tracker'").collect()) == 1):
      spark.sql("drop table " + entity + "_tracker")

In [19]:
resetTrackingData("landlord")

In [20]:
spark.sql("drop table landlord_data")

In [21]:
def getLastLandlordState_df():
  entity = "landlord"
  delta_df = getDelta_df(entity).drop("fetch_time")
  temp_state_df = ( delta_df.groupBy("Landlord_id").agg(F.max(delta_df.event_time))
                   .select(col("Landlord_id").alias("Landlord_id1"), col("max(event_time)").alias("event_time1")))
  delta_state_df = ( delta_df.join(temp_state_df,(delta_df.Landlord_id == temp_state_df.Landlord_id1) 
                                                & (delta_df.event_time == temp_state_df.event_time1))
                    .drop("Landlord_id1")
                    .drop("event_time1"))
  
  return delta_state_df

In [22]:
def replaceLandlordRows(update_df):
  for row in update_df.collect():
    spark.sql("delete from landlord_data where Landlord_id=" + str(row.Landlord_id))
  update_df.write.mode("append").saveAsTable("landlord_data")

In [23]:
def updateLandlord(new_State_df):
  
  spark.sql("create table if not exists landlord_data (Landlord_id int, Password string, Landlord_name string, Address_line_1 string, City string, Post_code string, Region string, event_time timestamp)")
  
  landlord_data_df = (spark.sql("select * from landlord_data")
                      .select(col("Landlord_id").alias("Landlord_id1"), 
                              col("Password").alias("Password1"), 
                              col("Landlord_name").alias("Landlord_name1"), 
                              col("Address_line_1").alias("Address_line_11"), 
                              col("City").alias("City1"), 
                              col("Post_code").alias("Post_code1"), 
                              col("Region").alias("Region1"), 
                              col("event_time").alias("event_time1")))
  
  update_rows_df = (landlord_data_df.join(state_df, (new_State_df.Landlord_id == landlord_data_df.Landlord_id1),  'outer')
             .select(new_State_df.Landlord_id, 
                     F.when(new_State_df.event_time > landlord_data_df.event_time1, new_State_df.Password)
                     .otherwise(landlord_data_df.Password1).alias("Password"), 
                     
                     F.when(new_State_df.event_time > landlord_data_df.event_time1, new_State_df.Landlord_name)
                     .otherwise(landlord_data_df.Landlord_name1).alias("Landlord_name"), 
                     
                     F.when(new_State_df.event_time > landlord_data_df.event_time1, new_State_df.Address_line_1)
                     .otherwise(landlord_data_df.Address_line_11).alias("Address_line_1"), 
                     
                     F.when(new_State_df.event_time > landlord_data_df.event_time1, new_State_df.City)
                     .otherwise(landlord_data_df.City1).alias("City"), 
                     
                     F.when(new_State_df.event_time > landlord_data_df.event_time1, new_State_df.Post_code)
                     .otherwise(landlord_data_df.Post_code1).alias("Post_code"), 
                     
                     F.when(new_State_df.event_time > landlord_data_df.event_time1, new_State_df.Region)
                     .otherwise(landlord_data_df.Region1).alias("Region"), 
                     
                     F.when(new_State_df.event_time > landlord_data_df.event_time1, new_State_df.event_time)
                     .otherwise(landlord_data_df.event_time1).alias("event_time")))
  
  new_ids_df = (new_State_df.select("Landlord_id").subtract(update_rows_df.select("Landlord_id"))
                .distinct().select(col("Landlord_id").alias("Landlord_id1")))
  
  new_rows_df = (new_State_df.join(new_ids_df, (new_State_df.Landlord_id == new_ids_df.Landlord_id1), "outer")
                 .drop("Landlord_id1")
                 .distinct())
  
  new_rows_df.write.insertInto("landlord_data")
  
#   replaceLandlordRows(update_rows_df)

In [24]:
state_df = getLastLandlordState_df()

In [25]:
display(state_df.orderBy("Landlord_id"))